In [3]:
import torch
import torch.optim as optim
import numpy as np
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# import torch
# import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim1),
            nn.ReLU(),
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.ReLU(),
            nn.Linear(hidden_dim2, output_dim)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# Load embeddings (shape: [n_samples, embedding_dim])
embeddings = torch.load('../clean_embeddings.pt')

# Placeholder for labels (modify this according to your task)
# Assume that y contains your labels
labels = torch.load('../clean_labels.pt').long()

# Convert to PyTorch tensors
# embeddings_tensor = torch.tensor(embeddings, dtype=torch.float32)
# labels_tensor = torch.tensor(labels, dtype=torch.long)  # Change to the appropriate dtype for regression tasks

# Create dataset and dataloader
dataset = TensorDataset(embeddings, labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [ ]:
# Define the MLP configuration
embedding_dim = embeddings.shape[1]
hidden_dim1 = 256
hidden_dim2 = 256
output_dim = 180  # Change this accordingly for regression tasks

mlp_model = MLP(embedding_dim, hidden_dim1, hidden_dim2, output_dim)
criterion = nn.CrossEntropyLoss()  # Use MSELoss or other appropriate loss for regression
optimizer = optim.Adam(mlp_model.parameters(), lr=1e-3)

# Training loop
num_epochs = 20

mlp_model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (embeddings_batch, labels_batch) in enumerate(dataloader):
        embeddings_batch, labels_batch = embeddings_batch.to(device), labels_batch.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = mlp_model(embeddings_batch)
        
        # Compute loss
        loss = criterion(outputs, labels_batch)
        running_loss += loss.item()
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}")

print("Training complete.")